In [1]:
import pandas as pd
import numpy as np
import sys, os
import gzip, json

In [3]:
df_hawaii = pd.read_json("/Users/hyoon/Desktop/review-Hawaii_10.json", lines=True)
df_miss = pd.read_json("/Users/hyoon/Desktop/review-Mississippi_10.json", lines=True)

In [4]:
df_hawaii_meta = pd.read_json("/Users/hyoon/Desktop/meta-Hawaii.json", lines=True)
df_miss_meta = pd.read_json("/Users/hyoon/Desktop/meta-Mississippi.json", lines=True)

In [5]:
# concat
df_review = pd.concat([df_hawaii, df_miss])
df_metadata = pd.concat([df_hawaii_meta, df_miss_meta])

In [8]:
# only necessary columns for our policy
df_reviews_reduced = df_review[["user_id", "time", "rating", "text", "gmap_id"]].copy()
df_metadata_reduced = df_metadata[["gmap_id", "name", "category", "description"]].copy()

In [9]:
df_merged = pd.merge(df_reviews_reduced, df_metadata_reduced, on="gmap_id", how="left")
df_merged

,user_id,time,rating,text,gmap_id,name,category,description
0,1.139654e+20,1591839903487,5,Great new upgrade,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
1,1.139654e+20,1591839903487,5,Great new upgrade,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
2,1.166558e+20,1568059018979,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
3,1.166558e+20,1568059018979,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
4,1.008341e+20,1594885588335,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
...,...,...,...,...,...,...,...,...
3477953,1.106579e+20,1567988614027,5,None,0x889c0976154e7d57:0xe745f61f5f4153e9,Coastal Magpie,[Antique store],None
3477954,1.017050e+20,1590511714469,2,None,0x889c0976154e7d57:0xe745f61f5f4153e9,Coastal Magpie,[Antique store],None
3477955,1.100881e+20,1561741287414,4,None,0x889c0976154e7d57:0xe745f61f5f4153e9,Coastal Magpie,[Antique store],None
3477956,1.061668e+20,1599430526156,5,None,0x889c0976154e7d57:0xe745f61f5f4153e9,Coastal Magpie,[Antique store],None


In [10]:
#rename columns, create new columsn, drop unneccesary columns
df_merged_processed = df_merged.rename(columns={"name":"business_name", "category":"business_category", "description":"business_description"})
df_merged_processed["_id"] = df_merged_processed["user_id"].astype(str) + "_" + df_merged_processed["time"].astype(str)
df_merged_processed = df_merged_processed.drop(columns=["user_id", "time", "gmap_id"])
df_merged_processed

,rating,text,business_name,business_category,business_description,_id
0,5,Great new upgrade,SMP - Single Marine Program,[Recreation center],None,1.1396541707957663e+20_1591839903487
1,5,Great new upgrade,SMP - Single Marine Program,[Recreation center],None,1.1396541707957663e+20_1591839903487
2,5,None,SMP - Single Marine Program,[Recreation center],None,1.1665581913729334e+20_1568059018979
3,5,None,SMP - Single Marine Program,[Recreation center],None,1.1665581913729334e+20_1568059018979
4,5,None,SMP - Single Marine Program,[Recreation center],None,1.0083411999455008e+20_1594885588335
...,...,...,...,...,...,...
3477953,5,None,Coastal Magpie,[Antique store],None,1.1065792427359904e+20_1567988614027
3477954,2,None,Coastal Magpie,[Antique store],None,1.0170500614791163e+20_1590511714469
3477955,4,None,Coastal Magpie,[Antique store],None,1.1008809657821312e+20_1561741287414
3477956,5,None,Coastal Magpie,[Antique store],None,1.06166803830674e+20_1599430526156


In [11]:
# remove duplicate _id and rows with null values or non english character
df_merged_drop_dup = df_merged_processed.drop_duplicates(subset=["_id"])
df_merged_final = df_merged_drop_dup[~df_merged_drop_dup.isnull().any(axis=1)]

allowed_pattern = r'^[A-Za-z0-9\s.,!?;:\'\"()\[\]\{\}<>-]*$'
df_merged_final = df_merged_final[(df_merged_final['text'].str.match(allowed_pattern, na=False))
                                  &(df_merged_final['business_name'].str.match(allowed_pattern, na=False))
                                  #&(df_merged_final['business_category'].str.match(allowed_pattern, na=False))
                                  &(df_merged_final['business_description'].str.match(allowed_pattern, na=False))]
df_merged_final

,rating,text,business_name,business_category,business_description,_id
180,5,My husband took me here for my birthday! The ...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.089786951023629e+20_1507964592497
182,4,was a great place. Now closed. Too bad. We'll...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.124085651879834e+20_1519880201044
184,5,"cozy, great food. love how you can sign your b...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1571615809204435e+20_1490033397528
186,4,"Not your average little Italian joint, don't e...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.181306411818766e+20_1511165172600
188,3,Service was great and the mozzarella fritta an...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1430211684992369e+20_1505179108178
...,...,...,...,...,...,...
3451581,4,Like,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.1299413904635706e+20_1569585534937
3451582,5,Good,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.007731860387506e+20_1552744512443
3451583,5,Awesome,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.0795762373548025e+20_1545454653879
3451584,3,Was ok,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.0893062348932448e+20_1553276365557


In [12]:
# filter rows with text less than 20 characters
df_merged_final_filtered = df_merged_final[df_merged_final["text"].str.len()>=20].copy()
df_merged_final_filtered

,rating,text,business_name,business_category,business_description,_id
180,5,My husband took me here for my birthday! The ...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.089786951023629e+20_1507964592497
182,4,was a great place. Now closed. Too bad. We'll...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.124085651879834e+20_1519880201044
184,5,"cozy, great food. love how you can sign your b...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1571615809204435e+20_1490033397528
186,4,"Not your average little Italian joint, don't e...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.181306411818766e+20_1511165172600
188,3,Service was great and the mozzarella fritta an...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1430211684992369e+20_1505179108178
...,...,...,...,...,...,...
3451572,2,Price is way too that's why I went to Domino's,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.147057364757655e+20_1565096616861
3451573,3,Have some ruled workers make u wait until they...,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.1163776495564964e+20_1518322261065
3451576,5,Awesome place to carryout,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.1285216373890079e+20_1548359544066
3451577,1,Took a hour to get pizza order over the phone,Pizza Hut,"[Pizza delivery, Chicken wings restaurant, Tak...",Family-friendly chain known for its made-to-or...,1.095308484207056e+20_1551372657942


In [15]:
df_merged_final_filtered.to_csv("final_data.csv", index=False)

In [13]:
# #extract N reviews per business_name for finalized train_validation dataset of 10000 entries.
# total_samples = 10000
# n_groups = df_merged_final_filtered['business_name'].nunique()
# samples_per_group = total_samples // n_groups

# df_sampled = df_merged_final_filtered.groupby('business_name', group_keys=False).apply(
#     lambda x: x.sample(n=min(len(x), samples_per_group), random_state=42)
# )

# if len(df_sampled) < total_samples:
#     remaining = total_samples - len(df_sampled)
#     df_remaining = df_merged_final_filtered.drop(df_sampled.index).sample(n=remaining, random_state=42)
#     df_sampled = pd.concat([df_sampled, df_remaining])

# print(len(df_sampled))
# print(df_sampled['business_name'].value_counts())

In [ ]:
# # split final_data.csv by 10000 entries each.
# df = pd.read_csv("final_data.csv")
# for i, v in enumerate(range(0,80000,10000)):
#     df_split = df.iloc[v:v+10000].copy()
#     #print(i+1)
#     #print(f"final_data_{i+1}.csv")
#     df_split.to_csv(f"final_data_{i+1}.csv")